# "First steps in regression (OLS and Gradient descent)"


- toc: false
- branch: master
- badges: true
- comments: true
- categories: [jupyter]

## Exploring regression

In this notebook, I want to start exploring regression. While python libraries provide many convenient ways to run a regression on a dataset, I want to start with a manual approach. The intent here is to cement my understanding of this topic

In [1]:
#importing
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
#I will use one of the statsmodels API, not sure which yet
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
from IPython.display import display_html, HTML
from io import StringIO

I am going to generate a simple linear dataset with one variable. I will then apply the following methods to arrive at the relationship of y with x  
1) Using the Ordinary Least Squares Method  
2) Using Gradient descent

In [2]:
#This generator function will generate data simple linear datasets
def genData(n = 20, slope = 1, intercept = 2, noise = False):
    x = np.linspace(1,n,n)
    y = slope*x + intercept
    if noise:
        y += np.random.normal(0,1,n)
    dataset = pd.DataFrame({'x':x, 'y': y})
    return x, y, dataset


In [3]:
#Lets start by using OLS in stats model to run a regression
slope = 2.5
intercept = -0.65
_,_,dataset = genData(n = 20, slope = slope, intercept = intercept, noise = True)

linear_regression = smf.ols(formula = 'y ~ x', data = dataset).fit()
linear_regression.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     6976.
Date:                Thu, 13 May 2021   Prob (F-statistic):           9.20e-25
Time:                        08:08:22   Log-Likelihood:                -21.939
No. Observations:                  20   AIC:                             47.88
Df Residuals:                      18   BIC:                             49.87
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.6273      0.355     -1.768      0.094      -1.373       0.118
x              2.4743      0.030     83.523      0.000       2.412       2.536
==============================================================================
Omnibus:                        2.672   Durbin-Watson:                   2.779
Prob(Omnibus):                  0.263   Jarque-Bera (JB):                1.733
Skew:                           0.720   Prob(JB):                        0.420
Kurtosis:                       2.940   Cond. No.                         25.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [4]:
#Lets now plot this relationship on a graph to get a visual understanding of the data
#using HTML to display plotly in fastpages
fig = go.Figure()
fig.add_trace(go.Scatter(x = dataset.x, y = dataset.y, mode='markers', name="Original Data"))
fig.add_trace(go.Scatter(x = dataset.x, y = linear_regression.predict(dataset.x), line_color = 'orange', name="Regression Line"))
HTML(fig.to_html(include_plotlyjs='cdn'))

THe OLS formula is well know and is used to miniminse SSE (sum of squared errors). Starting with the linear regression formula:

$$\hat{y}_i = \beta_0 + \beta_1x_i$$

In this expression $\hat{y}_i$ is the estimate for the true $y_i$. Our task is to find $\beta_0$ and $\beta_1$ that minimises the mean of the squared errors *(MSE)* given by the following formula

$$MSE = \frac{1}{n}\displaystyle\sum_{i=1}^{n} (y_i - \hat{y}_i)^2$$

Substituting $\hat{y}_i$ we get

$$MSE = \frac{1}{n}\displaystyle\sum_{i=1}^{n} (y_i - \beta_0 - \beta_1x_i)^2$$

Taking the partial derivatives of the equation above with respect to $\beta_0$ and $\beta_1$ and setting them to zero, we get the following

$$\beta_0 = \bar{y} - \beta_1\bar{x}$$  

$$\beta_1 = \frac{\displaystyle\sum_{i=1}^{n}(x_i - \bar{x})(y_i - \bar{y})}{\displaystyle\sum_{i=1}^{n} (x_i - \bar{x})^2} = \rho_{xy}\frac{\sigma_y}{\sigma_x}$$

*Note: [One of the many sources](https://towardsdatascience.com/understanding-the-ols-method-for-simple-linear-regression-e0a4e8f692cc)*

In [5]:
#Defining a function that returns beta0 and beta1 for a given x and y arrays
def OLS(x,y):
    rhoxy = np.corrcoef(y, x)
    sigmax = np.std(x)
    sigmay = np.std(y)
    beta1 = rhoxy[0,1]*(sigmay/sigmax)
    beta0 = np.mean(y) - beta1*(np.mean(x))
    return beta0, beta1
result = OLS(dataset.x, dataset.y)
print ("Beta0 = %0.4f" %result[0])
print("Beta1 = %0.4f" %result[1])


Beta0 = -0.6273
Beta1 = 2.4743


The values calculated by this formula match with the results of the statsmodel linear regression.

Now onto the fun part *__Gradient Descent__*

### Gradient Descent

There are tons of resources available that do a great job of explaining gradient descent. In summary, gradient descent is an iterative method that allows us to minimize the loss function. It may not be the best method for a single predictor linear regression since we have a well defined formula to calculate the cofficients. However for other applications there are loss functions that are more complex. In those situations, gradient descent makes the job faster.

Our goal with gradient descent is to traven down a gradient of the loss function and arrive at the _hopefully_ global minimum of the loss function. It may be the case that gradient descent will arrive at a gobal minimum. However as I understand it, for $MSE$ in linear regression, there is only 1 global minimum.

We will start by plotting the $MSE$ function to evaluate its shape. To do that, we have to calculate the value of the $MSE$ for various values of $\beta_0$ and $\beta_1$

In [6]:
from sklearn.metrics import mean_squared_error
#For a given range of b0 and b1, this function will return a 2-d array of the SSE
def mse_calc(y, x, interceptRange, slopeRange, size=20):
    intercept = np.linspace(interceptRange[0],interceptRange[1],size)
    slope = np.linspace(slopeRange[0],slopeRange[1],size)
    obs = len(y)
    z_vals = np.zeros((size,size))
    #Calculating the SSE for each combination of slope and intercept. TODO: Vectorize this
    for i in range(0,size):
        for j in range(0,size):
            y_pred = np.multiply(slope[j],x) + intercept[i]
            z_vals[i][j] = mean_squared_error(y, y_pred)
    return intercept, slope, z_vals

In [8]:
#Lets plot a contour map of the MSE to get a visual understanding of the problem that we have to solve
print("Results from OLS")
print("Intercept = ", round(result[0],2))
print("Slope = ", round(result[1],2))


interceptRange = [intercept - 2, intercept + 2]
slopeRange = [slope - 3,slope + 3]
surface_x, surface_y, surface_z = mse_calc(y = dataset.y, x = dataset.x, interceptRange = interceptRange,
                                           slopeRange = slopeRange, size = 50)
fig = go.Figure(data = [go.Contour(z = surface_z, x = surface_x, y = surface_y, colorscale = 'Bluyl')])
fig.add_trace(go.Scatter(x = [result[0]], y = [result[1]], mode='markers', hoverinfo = 'skip'))
fig.update_layout(
    title="Contour map of loss function",
    xaxis_title="Intercept",
    yaxis_title="Slope")
HTML(fig.to_html(include_plotlyjs='cdn'))



Results from OLS
Intercept =  -0.63
Slope =  2.47


Now that we have visualised the loss function, the next step is to implement our gradient descent algorithm and plot its path to understand how it works. *Note: [Great explanation here](https://towardsdatascience.com/linear-regression-using-gradient-descent-97a6c8700931)*

As discussed, gradient descent is an iterative formula which works by progressively moving in the direction of a local/global mimina of the loss function. In the case of linear regression with 1 predictor ($\hat{y}_i = \beta_0 + \beta_1x_i$), we would need to calculate the partial detivative of the loss function with respect to $\beta_0$ and $\beta_1$. We will then use a learning rate $\alpha$ to calculate the new value of $\beta_0$ and $\beta_1$. In this case, the partial derivatives are:

$$ \frac{\partial\beta_0}{\partial MSE} = \frac{-2}{n}\displaystyle\sum_{i=1}^{n} (y_i - \hat{y}_i)$$  
$$ \frac{\partial\beta_1}{\partial MSE} = \frac{-2}{n}\displaystyle\sum_{i=1}^{n} x_i(y_i - \hat{y}_i)$$

We start with randomly selected values of $\beta_0$ and $\beta_1$, generate $\hat{Y}$ and then use the following to update the values:

$$\beta_0 = \beta_0 - \alpha*\frac{\partial\beta_0}{\partial MSE}$$  
$$\beta_1 = \beta_1 - \alpha*\frac{\partial\beta_1}{\partial MSE}$$  

We stop when either of the following conditions is met:
1) The sucessive differences between the iterations fall below a treshold  
2) **OR** We have iterated an n number of times *(this n us user defined and I dont know of any rules of thumb here yet)*

I will start with a function that calculates this for us. Then I will plot this for an intutive understading

In [9]:
def gradient_descent(y,x,epochs = 10, treshold = 0.001, learning_rate = 0.0001, beta0 = 3, beta1 = 1):
    obs = float(len(y))
    learning_rate = learning_rate*(-2/obs)
    beta0path = [beta0]
    beta1path = [beta1]
    for i in range(0,epochs):
        y_pred = beta1*x + beta0
        beta0_adj = learning_rate*np.sum(y - y_pred)
        beta0 = beta0 - beta0_adj
        #For large observations, this equation causes an overflow. I have tried to figure it out, but have not
        #found a robust way of managing it. This works fine for ~20 obs. So it will do for our purposes
        beta1_adj = learning_rate*np.sum(x*(y - y_pred))
        beta1 = beta1 - beta1_adj
        beta0path.append(beta0)
        beta1path.append(beta1)
       
    return beta0, beta1, np.array(beta0path), np.array(beta1path)
    
print("Intercept (x) = ",result[0])
print("Slope (y) = ", result[1])

Intercept (x) =  -0.6273023448439652
Slope (y) =  2.4742514126098554


In [98]:
#x, y, _ = genData(n = 20, slope = 54, intercept = 0.456)
beta0, beta1, beta0path, beta1path = gradient_descent(dataset.y,dataset.x,epochs = 3500, 
                                                      learning_rate = 0.005, beta0 = -1.5, beta1 = 4)


In [99]:
fig = go.Figure(data = [go.Contour(z = surface_z, x = surface_x, y = surface_y, colorscale = 'Bluyl')])
fig.add_trace(go.Scatter(x = beta0path, y = beta1path, mode='lines'))
fig.add_hline(y = result[1] ,line_dash = 'dash', line_width = 0.5, line_color = "Blue")
fig.add_vline(x = result[0] ,line_dash = 'dash', line_width = 0.5, line_color = "Blue")
fig.update_layout(
    title="Contour map of loss function",
    xaxis_title="Intercept",
    yaxis_title="Slope")
fig.update_xaxes(range=interceptRange)
fig.update_yaxes(range=slopeRange)
fig.show()


The chart above shows the path gradient descent takes to 'arrive at the solution. The other variable of interest in gradient descent is $\alpha$ (the learning rate). In my experimentation so far, I have observed a high sensitivity to this variable. Too low and we dont arrive at a solution, too high and it jumps around and misses the solution. The charts below show how different learning rates impact the progression of the variables to the true value

In [220]:
from plotly.subplots import make_subplots
intercept = 2.5
slope = 4.5
epochs = 1200
x, y, dataset = genData(n = 20, slope = slope, intercept = intercept)
beta0guess = 0
beta1guess = -10
learningRates = [0.0001,0.0065, 0.007]
beta0results = np.zeros((3, epochs+1))
beta1results = np.zeros((3, epochs+1))
#Step 1: Generate learning paths for 3 different learning rates: Too cold, just right and too hot
_, _, beta0results[0], beta1results[0] = gradient_descent(dataset.y,dataset.x,epochs = epochs, 
                                         learning_rate = learningRates[0], beta0 = beta0guess, beta1 = beta1guess)

_, _, beta0results[1], beta1results[1] = gradient_descent(dataset.y,dataset.x,epochs = epochs, 
                                         learning_rate = learningRates[1],  beta0 = beta0guess, beta1 = beta1guess)

_, _, beta0results[2], beta1results[2] = gradient_descent(dataset.y,dataset.x,epochs = epochs, 
                                         learning_rate =  learningRates[2],  beta0 = beta0guess, beta1 = beta1guess)

#fig = go.Figure()
#fig = go.Scatter(x = np.arange(0,3500,1), y = beta0pathR, mode='lines')
#fig.add_vline(x = result[0],line_dash = 'dash', line_width = 0.5, line_color = "Blue")
#fig.show()


In [221]:
plttitles = ("Learning Rate = " + str(learningRates[0]), "Learning Rate = " + str(learningRates[0]),
            "Learning Rate = " + str(learningRates[1]), "Learning Rate = " + str(learningRates[1]),
            "Learning Rate = " + str(learningRates[2]), "Learning Rate = " + str(learningRates[2]))                                                                          
             
fig = make_subplots(rows=3, cols=2, shared_xaxes = True, subplot_titles = plttitles)
colors = ['blue','green', 'gray']
names = ['Cold', 'Just Right', 'Hot']
for i in [1,2]:
    for j in [1,2,3]:
        if i == 1:
            fig.add_trace(go.Scatter(x = np.arange(0,epochs,1), y = beta0results[j-1], mode='lines'
                                     , marker_color = colors[j-1], name = names[j-1]+' beta0'), row = j, col = i)
            fig.add_hline(y = intercept,line_dash = 'dash', line_width = 1, line_color = "red", 
                          row = j, col = i)
            fig.update_yaxes(range=[beta0guess, intercept + 0.5], row = j, col = i)    
        else:
            fig.add_trace(go.Scatter(x = np.arange(0,epochs,1), y = beta1results[j-1], mode='lines'
                                     , marker_color = colors[j-1], name = names[j-1]+' beta1'), row = j, col = i)
            fig.add_hline(y = slope,line_dash = 'dash', line_width = 1, line_color = "red", row = j, col = i)
            fig.update_yaxes(range=[beta1guess, slope + 5], row = j, col = i)    
fig.update_layout(height = 1000,
                 title_text="Impact of learning rate on gradient descent solution convergence")
fig.show()

As seen in the graphs above, in the gradient descent model, the learning rate impacts how the algorithm converges to a solution. Of particular interest is the Just right vs the Hot scenario. In the Just Right scenario, the algorithm converges to a solution. However a minute increase of 0.0005 in the learning rate causes the gradient descent model to vary widely and not converge to a solution.

What I did not expect was the difference in convergence behaviour between $\beta_0$ and $\beta_1$. In the Cold scenario, $\beta_0$ does not converge, however $\beta_1$ conerges to a solution.

This concludes my initial exploration of regression. In subsequent notebooks, I will apply my new found knowledge to more practical problems.